In [6]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

True

### **Runnables**

##### **1) Runnable Sequence-**  
RunnableSequence is a built-in primitive in LangChain that lets you connect multiple Runnables in a step-by-step (sequential) order.

In [7]:
from langchain_core.runnables import RunnableSequence

model = ChatGroq(model="llama-3.3-70b-versatile")

prompt1 = PromptTemplate(
    template = "Write a joke about {topic}",
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template = "Explain the joke to me like I'm 5: {joke}",
    input_variables=['joke']
)

parser = StrOutputParser()

chain1 = RunnableSequence(prompt1, model, parser)
print(chain1.invoke({'topic': "AI"}))

chain2 = RunnableSequence(prompt1, model, parser, prompt2, model, parser)
print(chain2.invoke({'topic': 'AI'}))

Why did the AI program go to therapy?

Because it was struggling to process its emotions! (get it?)
Oh boy, let me explain the joke to you!

So, you know how we have feelings like happy, sad, and mad? And sometimes we need to talk to someone to help us feel better?

Well, a computer program called AI (say "Aye-Eye") is like a super smart robot that can do lots of things. But it's not a real person, so it doesn't have feelings like we do.

The joke is saying that the AI program went to therapy, which is like a special kind of help for people who need to talk about their feelings. But the AI program can't really have feelings like we do, because it's just a computer!

The funny part is that the joke says the AI program is "struggling to process its emotions". "Process" is a big word that means "to think about or understand something". And computers are really good at processing things, like numbers and words!

So, the joke is making a silly pun (that's a joke that uses words in a funny w

##### **2) Runnable Parallel**
allows you to run multiple tasks at the same time (in parallel) using the same input.

In [8]:
from langchain_core.runnables import RunnableParallel

model = ChatGroq(model="llama-3.3-70b-versatile")

prompt1 = PromptTemplate(
    template = "Generate a tweet about {topic}",
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template="Generate a Linkdin post about {topic}",
    input_variables=['topic']
)

parser = StrOutputParser()

parallel_chain = RunnableParallel({
    "tweet": prompt1 | model | parser,
    "Linkdin": prompt2 | model | parser
})

result = parallel_chain.invoke({'topic': 'AI'})

print(result['tweet'])
print(result['Linkdin'])

"AI is revolutionizing the way we live & work! From virtual assistants to self-driving cars, the possibilities are endless. What's the most exciting AI innovation you're looking forward to? #AI #ArtificialIntelligence #FutureTech"
Here's a potential LinkedIn post about AI:

**The Future is Here: Harnessing the Power of AI**

As we continue to navigate the ever-changing landscape of technology, one thing is clear: Artificial Intelligence (AI) is revolutionizing the way we work, live, and interact with one another.

From automating routine tasks to analyzing complex data sets, AI is transforming industries and opening up new opportunities for innovation and growth. But what does this mean for us as professionals, and how can we harness the power of AI to drive success in our careers and organizations?

Here are a few key takeaways to consider:

 **Upskilling is key**: As AI takes over routine tasks, it's essential to develop skills that complement AI, such as critical thinking, creativit

##### **Runnable Passthrough**
RunnablePassthrough is a special type of Runnable in LangChain that returns the exact input as the output without any processing.

In [18]:
from langchain_core.runnables import RunnablePassthrough

model = ChatGroq(model="llama-3.3-70b-versatile")

parser = StrOutputParser()

Prompt1 = PromptTemplate(
    template= "Write a joke about {topic}",
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Explain the joke to me like I\'m 5: {joke}',
    input_variables=['joke']
)

joke_gen = RunnableSequence(prompt1, model, parser)

parallel_chain = RunnableParallel({
    "joke": RunnablePassthrough(),
    "explaination": RunnableSequence(prompt2, model, parser)
})

final_chain = joke_gen | parallel_chain

result = final_chain.invoke({'topic': 'Food'})
print(result)

{'joke': "**Comprehensive Report on Food**\n\n**Introduction**\n\nFood is a vital component of human life, providing the necessary nutrients, energy, and sustenance for growth, development, and maintenance of overall health. The food industry is a complex and multifaceted sector that encompasses various aspects, including production, processing, distribution, and consumption. This report aims to provide an in-depth analysis of the food industry, covering its definition, types, production, processing, distribution, consumption, and impact on health and the environment.\n\n**Definition and Types of Food**\n\nFood refers to any substance consumed by humans or animals to provide nutritional support and energy. There are several types of food, including:\n\n1. **Fruits**: Fruits are the sweet and fleshy parts of plants that grow from flowers and contain seeds. Examples include apples, bananas, and berries.\n2. **Vegetables**: Vegetables are the edible parts of plants, including leaves, stem

##### **Runnable Lambda**
RunnableLambda lets you convert any custom Python function into a Runnable, so it can be used inside LangChain pipelines like other components (e.g., LLMs, prompt templates, etc.).

In [15]:
from langchain_core.runnables import RunnableLambda

model = ChatGroq(model="llama-3.3-70b-versatile")
parser = StrOutputParser()

prompt1 = PromptTemplate(
    template="Write a joke about {topic}",
    input_variables={'topic'}
)

def word_counter(text):
    return len(text.split())

joke_gen = prompt1 | model | parser

parallel_chain = RunnableParallel({
    "joke": RunnablePassthrough(),
    "word_count": RunnableLambda(word_counter)  # We can also write RunnableLambda(lambda x: len(x.split()))
})

final_chain = joke_gen | parallel_chain

result = final_chain.invoke({'topic': 'AI'})
print(result)

{'joke': 'Why did the AI program go to therapy?\n\nBecause it was struggling to process its emotions! (get it?)', 'word_count': 18}


##### **Runnable Branch**
RunnableBranch is like an if-else logic for LangChain pipelines. It helps you conditionally run different chains or actions based on the input or model output.

In [ ]:
from langchain_core.runnables import RunnableBranch

model = ChatGroq(model="llama-3.3-70b-versatile")
parser= StrOutputParser()

prompt1 = PromptTemplate(
    template = "Generate a detailed report about the {topic}",
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template = "Generate a summary of the {text}",
    input_variables=['text']
)

report_gen = prompt1 | model | parser

branch_chain =RunnableBranch(
    (lambda x: len(x.split()) > 500, prompt2 | model | parser), # Summarizes the report if it's more than 500 words
    RunnablePassthrough()
)

final_chain = report_gen | branch_chain
print(final_chain.invoke({'topic': 'US Tariffs'}))

The introduction to the report provides an overview of the current US tariff landscape, including the types of tariffs imposed, the affected countries and products, and the impact of these tariffs on the US economy and trade relationships. The report highlights that the United States has a long history of imposing tariffs to protect domestic industries, raise revenue, and influence trade policies. The introduction sets the stage for the report, which aims to provide a comprehensive analysis of the US tariff landscape and its effects on the economy and trade relationships. 

Key points from the introduction include:

* The US has a long history of imposing tariffs on imported goods
* Tariffs can have significant effects on international trade, economic growth, and consumer prices
* The report will provide an overview of the current US tariff landscape, including types of tariffs, affected countries and products, and the impact on the US economy and trade relationships. 

Overall, the in